<a href="https://colab.research.google.com/github/rishu-619/COMPUTER-VISION/blob/master/Topic_Modelling%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# download news group data set from sklearn
from sklearn.datasets import fetch_20newsgroups

In [5]:
#assign train and test data
ng_train = fetch_20newsgroups(subset = 'train',shuffle=True)
ng_test   = fetch_20newsgroups(subset = 'test',shuffle=True)

In [6]:
print(list(ng_train.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [7]:
#sample news
ng_train.data[:5]

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

In [8]:
#shape of the subsets
print(ng_train.filenames.shape,ng_train.target.shape)

(11314,) (11314,)


In [9]:
#Data Preprocessing
#Tokenization
'''
Loading gensim and nltk libraries
'''
#!pip install gensim
import gensim

# Converts into tokens (Alternative to word_tokenize)
from gensim.utils import simple_preprocess

from gensim.parsing.preprocessing import STOPWORDS

from nltk.stem import WordNetLemmatizer, SnowballStemmer
#from nltk.stem import *
import numpy as np
np.random.seed(400)

In [10]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [11]:
#testing the lemmatize statment. It should convert past or future tense into present tense
WordNetLemmatizer().lemmatize('runs')

'run'

In [12]:
#testing the stemming part before preprocessing. This should be changing any plural into singular word
import pandas as pd
stemmer = SnowballStemmer("english")
original_words=['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']

singles= [WordNetLemmatizer().lemmatize(plural,pos='v') for plural in original_words]

pd.DataFrame(data={'Original Words':original_words, 'Lemma':singles})

#Stemma is not performing well 
#singles= [stemmer.stem(plural) for plural in original_words]
#singles2= [stemmer.stem(plural2) for plural2 in singles]
#stemma = pd.DataFrame(data={'Lemma':singles, 'Stemmed':singles2})



,Original Words,Lemma
0,caresses,caress
1,flies,fly
2,dies,die
3,mules,mules
4,denied,deny
5,died,die
6,agreed,agree
7,owned,own
8,humbled,humble
9,sized,size


In [13]:
# writing function for the entire dataset
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))

#Tokenize and Lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>3:
            result.append(lemmatize_stemming(token))
            
    return result
        

In [14]:
#preview a document before preprocessing

#document_num = 50
doc_sample = 'This disk has failed many times. I would like to get it replaced.'

print("Original Document: ")
words=[]
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original Document: 
['This', 'disk', 'has', 'failed', 'many', 'times.', 'I', 'would', 'like', 'to', 'get', 'it', 'replaced.']


Tokenized and lemmatized document: 
['disk', 'fail', 'time', 'like', 'replac']


In [15]:
processed_docs = []

for doc in ng_train.data:
    processed_docs.append(preprocess(doc))

In [16]:
len(processed_docs)

11314

In [17]:
print(processed_docs[:2])

[['lerxst', 'thing', 'subject', 'nntp', 'post', 'host', 'organ', 'univers', 'maryland', 'colleg', 'park', 'line', 'wonder', 'enlighten', 'door', 'sport', 'look', 'late', 'earli', 'call', 'bricklin', 'door', 'small', 'addit', 'bumper', 'separ', 'rest', 'bodi', 'know', 'tellm', 'model', 'engin', 'spec', 'year', 'product', 'histori', 'info', 'funki', 'look', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst'], ['guykuo', 'carson', 'washington', 'subject', 'clock', 'poll', 'final', 'summari', 'final', 'clock', 'report', 'keyword', 'acceler', 'clock', 'upgrad', 'articl', 'shelley', 'qvfo', 'innc', 'organ', 'univers', 'washington', 'line', 'nntp', 'post', 'host', 'carson', 'washington', 'fair', 'number', 'brave', 'soul', 'upgrad', 'clock', 'oscil', 'share', 'experi', 'poll', 'send', 'brief', 'messag', 'detail', 'experi', 'procedur', 'speed', 'attain', 'rat', 'speed', 'card', 'adapt', 'heat', 'sink', 'hour', 'usag', 'floppi', 'disk', 'function', 'floppi', 'especi', 'request', 'summar', 'day',

In [18]:
#bag of words on the data set
dictionary = gensim.corpora.Dictionary(processed_docs)

In [19]:
print (dictionary)

Dictionary(61411 unique tokens: ['addit', 'bodi', 'bricklin', 'bring', 'bumper']...)


In [20]:
#Lets see if dictionary created succesfully
count=0
for k,v in dictionary.iteritems():
    print (k, v)
    count +=1
    if count >20:
        break

0 addit
1 bodi
2 bricklin
3 bring
4 bumper
5 call
6 colleg
7 door
8 earli
9 engin
10 enlighten
11 funki
12 histori
13 host
14 info
15 know
16 late
17 lerxst
18 line
19 look
20 mail


In [21]:
#remove rare and repeatative words
dictionary.filter_extremes(no_below=15,no_above=0.1,keep_n=100000)

In [22]:
print (dictionary)

Dictionary(6536 unique tokens: ['addit', 'bodi', 'bring', 'bumper', 'call']...)


In [23]:
#Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
#words and how many times those words appear. Save this to 'bow_corpus'
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [24]:
bow_corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 2),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1)]

In [25]:
#preview 
document_num = 10
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]],
                                                     bow_doc_x[i][1]))

Word 14 ("model") appears 1 time.
Word 33 ("clock") appears 1 time.
Word 46 ("keyword") appears 1 time.
Word 63 ("summari") appears 1 time.
Word 83 ("email") appears 1 time.
Word 101 ("opinion") appears 1 time.
Word 130 ("worth") appears 1 time.
Word 146 ("nice") appears 1 time.
Word 227 ("hard") appears 1 time.
Word 385 ("owner") appears 1 time.
Word 399 ("axi") appears 1 time.
Word 400 ("beemer") appears 1 time.
Word 401 ("bike") appears 2 time.
Word 402 ("brown") appears 1 time.
Word 403 ("ducati") appears 2 time.
Word 404 ("expir") appears 1 time.
Word 405 ("fade") appears 1 time.
Word 406 ("leak") appears 2 time.
Word 407 ("lonestar") appears 2 time.
Word 408 ("motor") appears 1 time.
Word 409 ("orang") appears 1 time.
Word 410 ("paint") appears 1 time.
Word 411 ("pop") appears 1 time.
Word 412 ("recommend") appears 1 time.
Word 413 ("richardson") appears 2 time.
Word 414 ("run") appears 1 time.
Word 415 ("sell") appears 1 time.
Word 416 ("shop") appears 1 time.
Word 417 ("stabl")

In [26]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 10, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [27]:
for idx, topic in lda_model.print_topics():
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.018*"game" + 0.015*"team" + 0.010*"play" + 0.010*"player" + 0.007*"season" + 0.007*"hockey" + 0.005*"leagu" + 0.005*"score" + 0.004*"bike" + 0.004*"basebal"


Topic: 1 
Words: 0.005*"weapon" + 0.005*"gun" + 0.004*"firearm" + 0.004*"effect" + 0.004*"drug" + 0.004*"crime" + 0.004*"caus" + 0.004*"food" + 0.004*"control" + 0.004*"pitt"


Topic: 2 
Words: 0.017*"wire" + 0.016*"scsi" + 0.008*"grind" + 0.007*"sandvik" + 0.007*"circuit" + 0.007*"connect" + 0.007*"appl" + 0.006*"kent" + 0.006*"militia" + 0.005*"outlet"


Topic: 3 
Words: 0.010*"space" + 0.008*"nasa" + 0.005*"presid" + 0.005*"nation" + 0.005*"govern" + 0.004*"center" + 0.004*"program" + 0.004*"orbit" + 0.003*"report" + 0.003*"research"


Topic: 4 
Words: 0.012*"encrypt" + 0.011*"govern" + 0.010*"israel" + 0.010*"secur" + 0.010*"chip" + 0.009*"isra" + 0.008*"clipper" + 0.006*"public" + 0.006*"key" + 0.005*"arab"


Topic: 5 
Words: 0.012*"toronto" + 0.009*"ohio" + 0.008*"columbia" + 0.008*"canada" + 0.007*"henri

In [28]:
num = 100
unseen_document = ng_test.data[num]
print(unseen_document)

Subject: help
From: C..Doelle@p26.f3333.n106.z1.fidonet.org (C. Doelle)
Lines: 13

Hello All!

    It is my understanding that all True-Type fonts in Windows are loaded in
prior to starting Windows - this makes getting into Windows quite slow if you
have hundreds of them as I do.  First off, am I correct in this thinking -
secondly, if that is the case - can you get Windows to ignore them on boot and
maybe make something like a PIF file to load them only when you enter the
applications that need fonts?  Any ideas?


Chris

 * Origin: chris.doelle.@f3333.n106.z1.fidonet.org (1:106/3333.26)



In [29]:
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 30)))

Score: 0.3599081039428711	 Topic: 0.017*"file" + 0.012*"program" + 0.008*"version" + 0.007*"card" + 0.007*"avail" + 0.007*"imag" + 0.007*"graphic" + 0.007*"window" + 0.006*"softwar" + 0.005*"color" + 0.005*"list" + 0.005*"send" + 0.005*"data" + 0.005*"email" + 0.005*"server" + 0.005*"info" + 0.005*"sourc" + 0.005*"user" + 0.005*"entri" + 0.005*"support" + 0.004*"internet" + 0.004*"address" + 0.004*"format" + 0.004*"video" + 0.004*"code" + 0.004*"type" + 0.004*"system" + 0.004*"build" + 0.004*"display" + 0.004*"unix"
Score: 0.12011542171239853	 Topic: 0.018*"drive" + 0.006*"price" + 0.006*"sale" + 0.006*"disk" + 0.006*"control" + 0.006*"speed" + 0.006*"hard" + 0.005*"engin" + 0.005*"power" + 0.005*"sell" + 0.005*"access" + 0.004*"appl" + 0.004*"buy" + 0.004*"high" + 0.004*"card" + 0.003*"driver" + 0.003*"sound" + 0.003*"model" + 0.003*"offer" + 0.003*"ship" + 0.003*"car" + 0.003*"instal" + 0.003*"cost" + 0.003*"board" + 0.003*"best" + 0.003*"monitor" + 0.003*"chip" + 0.003*"printer" + 0

In [ ]:
#bow_vector = dictionary.doc2bow(preprocess(unseen_document))

#for index, score in sorted(lda_model[bow_vector], key=lambda x: x[1]):
#    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

In [ ]:
#print(ng_test.target[1])